Part 1: Setup and Data Loading

Load Dataset 

In [ ]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

# Define source dataset directory
source_dataset = "tamil-dataset" 

# Create final_data directory
base = "final_data"

train = os.path.join(base, 'train')
test = os.path.join(base, 'test')
os.makedirs(train)
os.makedirs(test)

# Get class names from source dataset
classes = [d for d in os.listdir(source_dataset) if os.path.isdir(os.path.join(source_dataset, d))]

print(f"Found classes: {classes}")

all_images = []
all_labels = []

# Collect all image paths and labels
for class_name in classes:
    class_path = os.path.join(source_dataset, class_name)
    images = os.listdir(class_path)
    
    # Create class directories in train and test folders
    os.makedirs(os.path.join(train, class_name), exist_ok=True)
    os.makedirs(os.path.join(test, class_name), exist_ok=True)
    
    for img in images:
        if img.lower().endswith(('.tiff')):
            all_images.append(os.path.join(class_path, img))
            all_labels.append(class_name)

print(f"Total images found: {len(all_images)}")

print("Splitting data (80% Train, 20% Test)")

# Stratified split to maintain class distribution
X_train, X_test, y_train, y_test = train_test_split(
    all_images, 
    all_labels, 
    test_size=0.2, 
    stratify=all_labels,
    random_state=42
)

def copy_images(image_list, labels, destination_folder):
    for src_path, label in zip(image_list, labels):
        dst_path = os.path.join(destination_folder, label, os.path.basename(src_path))
        shutil.copy(src_path, dst_path)

print("Copying training images")
copy_images(X_train, y_train, train)

print("Copying test images")
copy_images(X_test, y_test, test)

print("\nData Split Summary:")
print(f"Training Images: {len(X_train)}")
print(f"Test Images:     {len(X_test)}")
print(f"Total:           {len(X_train) + len(X_test)}")

Part 2: Data Preview and Pipeline


Displaying random 3 sample of images from each class.

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

# Dataset path
train_dir = "final_data/train"
classes = os.listdir(train_dir)

plt.figure(figsize=(12, 8))
img_count = 1

for class_name in classes:
    class_path = os.path.join(train_dir, class_name)
    images = random.sample(os.listdir(class_path), 3)
    
    for img_name in images:
        img_path = os.path.join(class_path, img_name)
        # Load image using tf.keras.utils.load_img
        img = tf.keras.utils.load_img(img_path, color_mode="grayscale")
        # Convert to array
        img_array = tf.keras.utils.img_to_array(img)
        
        plt.subplot(len(classes), 3, img_count)
        plt.imshow(tf.squeeze(img_array), cmap="gray")
        plt.title(class_name)
        plt.axis("off")
        img_count += 1

plt.tight_layout()
plt.show()


Keras Data Generator

In [ ]:
import warnings
warnings.filterwarnings('ignore', message='.*Using ".tiff" files with multiple bands.*')

from tensorflow import keras
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator

# Define Generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=0.2
)

# Load Training Data
print("Loading Training Data:")
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Load Validation Data
print("\nLoading Validation Data:")
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

Part 3: Model Definition, Tuning, and Training

Define two models: a custom Simple CNN and a Transfer Learning Model
(MobileNetV2)

Simple CNN 

In [ ]:
import tensorflow as tf

def build_simple_cnn(input_shape=(64,64,1), num_classes=4):
    model = tf.keras.Sequential([
        tf.keras.Input(shape=input_shape),  # <-- input layer
        
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),

        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),

        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Instantiate
simple_cnn = build_simple_cnn()
simple_cnn.compile(
    optimizer='adam',                 
    loss='categorical_crossentropy',
    metrics=['accuracy'] 
)
simple_cnn.summary()


Transfer Learning Model (MobileNetV2)

In [ ]:
import tensorflow as tf

def build_transfer_model(num_classes=4):
    input_tensor = tf.keras.Input(shape=(64, 64, 1)) 
    
    # Resize up to 224x224
    x = tf.keras.layers.Resizing(224, 224)(input_tensor)
    
    # Convert 0-1 range to -1-1 range for MobileNet
    x = tf.keras.layers.Rescaling(scale=2.0, offset=-1.0)(x)
    
    # Convert grayscale to RGB
    x = tf.keras.layers.Lambda(lambda x: tf.image.grayscale_to_rgb(x))(x)
    
    # Load MobileNetV2 Model
    base_model = tf.keras.applications.MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3)
    )
    base_model.trainable = False

    # Connect Base Model
    x = base_model(x, training=False)
    
    # Add Classification Head
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)  # Added Dropout for safety
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    # Create Model
    model = tf.keras.models.Model(inputs=input_tensor, outputs=output)
    return model

# Instantiate
print("Building Transfer Learning Model...")
transfer_model = build_transfer_model(num_classes=4)

# Compile
transfer_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

transfer_model.summary()

HyperParameter Tuning

In [ ]:
import keras_tuner as kt
import tensorflow as tf

def build_hypermodel(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(64, 64, 1)))
    
    # Tuner decides: 32, 64, or 96 filters?
    hp_filters = hp.Int('filters', min_value=32, max_value=96, step=32)
    model.add(tf.keras.layers.Conv2D(hp_filters, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Flatten())
    
    # Tuner decides: 64, 128, or 192 units?
    hp_units = hp.Int('units', min_value=64, max_value=256, step=64)
    model.add(tf.keras.layers.Dense(hp_units, activation='relu'))
    
    model.add(tf.keras.layers.Dense(4, activation='softmax'))
    
    # Tuner decides: Learning rate?
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

tuner = kt.RandomSearch(
    build_hypermodel,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='tuner_log',
    project_name='cnn_tuning',
    overwrite=True
)

print("Searching for best hyperparameters...")
tuner.search(train_generator, epochs=5, validation_data=validation_generator)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\nSimple CNN Hyperparameters:")
print(f"{{'learning_rate': {best_hps.get('learning_rate')}, "
      f"'units': {best_hps.get('units')}, "
      f"'filters': {best_hps.get('filters')}}}")

# overwrite 'simple_cnn'
simple_cnn = tuner.hypermodel.build(best_hps)

print("\nSuccess! 'simple_cnn' has been tuning.")

Training

In [ ]:
import tensorflow as tf

cnn_log = tf.keras.callbacks.CSVLogger('model_cnn_training.log')
transfer_log = tf.keras.callbacks.CSVLogger('model_transfer_training.log')

# Train Simple CNN
print("Training Simple CNN...")
history_cnn = simple_cnn.fit(
    train_generator,
    epochs=15,                 # It will try to loop 15 times
    validation_data=validation_generator,
    callbacks=[cnn_log]
)
# Save it
simple_cnn.save('model_cnn.h5')


# Train Transfer Learning Model
print("\nTraining Transfer Learning Model...")
history_transfer = transfer_model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    callbacks=[transfer_log]
)
# Save it
transfer_model.save('model_transfer.h5')

print("\nDONE! Models are now trained and saved.")

Part 4: Model Evaluation and Prediction

Model Evaluation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    "final_data/test",
    target_size=(64, 64),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False 
)

def evaluate_model(model, model_name):
    print(f"\nEvaluating {model_name}:")
    
    loss, accuracy = model.evaluate(test_generator)
    print(f"{model_name} Test Accuracy: {accuracy * 100:.2f}%")
    
    test_generator.reset()
    predictions = model.predict(test_generator)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_generator.classes
    class_labels = list(test_generator.class_indices.keys())
    
    cm = confusion_matrix(true_classes, predicted_classes)
    
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_labels, yticklabels=class_labels)
    plt.title(f'Confusion Matrix: {model_name}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()
    
    return accuracy

acc_cnn = evaluate_model(simple_cnn, "Simple CNN")
acc_transfer = evaluate_model(transfer_model, "Transfer Model")

print("\nFinal Comparison")
print(f"Simple CNN Accuracy: {acc_cnn*100:.2f}%")
print(f"Transfer Model Accuracy: {acc_transfer*100:.2f}%")
best_model = transfer_model if acc_transfer > acc_cnn else simple_cnn
best_name = "Transfer Model" if acc_transfer > acc_cnn else "Simple CNN"
print(f"The winner is: {best_name}")

print(f"\nVisualizing 10 Samples from {best_name}:")

def visualize_10_samples(model):
    temp_gen = test_datagen.flow_from_directory(
        "final_data/test", target_size=(64, 64), batch_size=32, 
        color_mode='grayscale', class_mode='categorical', shuffle=True
    )
    images, labels = next(temp_gen)
    
    preds = model.predict(images)
    pred_classes = np.argmax(preds, axis=1)
    true_classes = np.argmax(labels, axis=1)
    class_names = list(test_generator.class_indices.keys())
    correct = np.sum(pred_classes[:10] == true_classes[:10])
    print(f"Accuracy over 10 samples: {correct}/10 ({correct*10}%)")

    plt.figure(figsize=(15, 6))
    for i in range(10):
        ax = plt.subplot(2, 5, i + 1)
        plt.imshow(images[i].squeeze(), cmap='gray')
        
        is_correct = pred_classes[i] == true_classes[i]
        color = 'green' if is_correct else 'red'
        
        plt.title(f"True: {class_names[true_classes[i]]}\nPred: {class_names[pred_classes[i]]}", color=color)
        plt.axis("off")
    plt.tight_layout()
    plt.show()

visualize_10_samples(best_model)

Prediction

In [ ]:
print("\nImage Provided Result:")

def predict_user_image(model, img_path):
    try:
        img = tf.keras.utils.load_img(img_path, target_size=(64, 64), color_mode='grayscale')
        
        img_array = tf.keras.utils.img_to_array(img)
        img_array = img_array / 255.0
        img_batch = np.expand_dims(img_array, axis=0)
        
        prediction = model.predict(img_batch)
        class_idx = np.argmax(prediction)
        
        class_labels = list(test_generator.class_indices.keys())
        predicted_label = class_labels[class_idx]
        confidence = prediction[0][class_idx]
        
        plt.figure(figsize=(4, 4))
        plt.imshow(img_array.squeeze(), cmap='gray')
        plt.title(f"Pred: {predicted_label} ({confidence*100:.1f}%)")
        plt.axis('off')
        plt.show()
        
    except Exception as e:
        print(f"Error: Could not read image at {img_path}. Check the path.")

img_path = input("Enter full path to an image to test")
if img_path:
    predict_user_image(best_model, img_path)